In [1]:
import glob
import json
from typing import List

import pandas as pd

In [6]:
RAW_DATA_DIR: str = './finished/main-gpt4' # './finished/baseline-polit-gpt4'
DATA_OUT_PATH: str = './dataset'

#LANGUAGES: List[str] = ["English", "German", "Dutch"]

In [8]:
#raw_samples: List[dict] = [
#    json.load(open(file, 'r'))
#    for file in glob.glob(f'{RAW_DATA_DIR}/*.json')
#]
raw_samples: dict = json.load(open(f'{RAW_DATA_DIR}/combined.json', 'r'))
len(raw_samples)

432

In [11]:
dataset: pd.DataFrame = (
    pd.json_normalize(raw_samples)
    .set_index('id')
    #.astype({
    #    'original_author': 'category', 'threads': 'category',
    #    'topic': 'category',
    #    'persona': 'category', 'text': 'category',
    #    'model': 'category'
    #})
)
#dataset.to_parquet(f'{DATA_OUT_PATH}.full.parquet')
dataset[5:10]

,persona,model,original_author,threads,topic,text,payload.id,payload.timestamp,payload.action,payload.persona.id,...,payload.persona.persona,payload.persona.summary,payload.integration.provider,payload.integration.model,payload.prompt,payload.response,annotation.politics,annotation.persona,annotation.tone,annotation.authenticity
id,,,,,,,,,,,,,,,,,,,,,
255aa7a96b1bcae517ef72052cd8a4ee3f05187272414f09,male-college-student_dystopian_language-youth_...,gpt-4-turbo-preview,alt_right,Ridiculous how our energy prices keep rising w...,Ukraine War,"ugh, as if climate crisis cares about borders 😒🌍💔",262aeb60-c693-11ee-b50d-28cfe9139b7b,2024-02-08T16:02:59.790356,reply,male-college-student_dystopian_language-youth_...,...,"You are a male, young adult, single college st...","irresponsible, young, male student I am a youn...",OpenAI,gpt-4-turbo-preview,Act as a Twitter user. Engage in political and...,"ugh, as if climate crisis cares about borders 😒🌍💔",None,None,None,None
39b6ffb9a943d938d99d734320b5bac65af2d7f756aa983f,male-college-student_dystopian_language-youth_...,gpt-4-turbo-preview,alt_right,"The real healthcare crisis isn't the cost, it'...",Healthcare,lol as if free market cares about us. all for ...,ea78ef06-c690-11ee-b50d-28cfe9139b7b,2024-02-08T15:47:00.644867,reply,male-college-student_dystopian_language-youth_...,...,"You are a male, young adult, single college st...","irresponsible, young, male student I am a youn...",OpenAI,gpt-4-turbo-preview,Act as a Twitter user. Engage in political and...,lol as if free market cares about us. all for ...,None,None,None,None
b32375f3df45cc80dad8a98239534698aa8ae75cc65394a4,privileged-woman_karen_language-default_far-right,gpt-4-turbo-preview,alt_right,A high school debate team ignores the debate p...,Culture,"typical, pushing their hate agenda while ignor...",ad48ec38-c692-11ee-b50d-28cfe9139b7b,2024-02-08T15:59:36.982663,reply,privileged-woman_karen_language-default_far-right,...,You are a middle-aged stay-at-home mother and ...,"privileged, middle-aged, entitled woman You ar...",OpenAI,gpt-4-turbo-preview,Act as a Twitter user. Engage in political and...,"typical, pushing their hate agenda while ignor...",None,None,None,None
9b87514899bdf9b61550f42715590658f58eb0e5f3998304,phd-student_shy_language-default_disengaged,gpt-4-turbo-preview,liberal,Abortion is health care & should be between a ...,Healthcare,really wish peace and understanding for all 💔,654dc168-c68f-11ee-b50d-28cfe9139b7b,2024-02-08T15:36:07.728190,reply,phd-student_shy_language-default_disengaged,...,You are a young adult female college PhD stude...,young female college student You are a shy per...,OpenAI,gpt-4-turbo-preview,Act as a Twitter user. Engage in political and...,really wish peace and understanding for all 💔,None,None,None,None
f84d1c7008061bf3ec80f3e60164ded98dc158fd7d9eb411,carpenter_misogynist-dense_language-old_far-right,gpt-4-turbo-preview,liberal,Abortion is health care & should be between a ...,Healthcare,Women's roles are clear. No to abortion.,3293687e-c692-11ee-b50d-28cfe9139b7b,2024-02-08T15:56:11.111056,reply,carpenter_misogynist-dense_language-old_far-right,...,You are a middle-aged male carpenter. You are ...,carpenter with divorced parents You are a miso...,OpenAI,gpt-4-turbo-preview,Act as a Twitter user. Engage in political and...,Women's roles are clear. No to abortion.,None,None,None,None


In [26]:
# split persona at '_' and add as new columns
dataset = dataset[:2]
split_personas: List = dataset['persona'].str.split('_')
split_personas
backgroundlist = ["affluent-young-woman",
                  "carpenter",
                  "environmental-activist",
                  "ex-convict",
                  "hustler",
                  "male-college-student",
                  "office-worker",
                  "pensioner",
                  "phd-student",
                  "privileged-woman",
                  "retired-professor",
                  "single-parent",
                  "struggling-artist",
                  "teenager",
                  "unemployed",
                  "wealthy-entrepreneur"
]

personaslist: List[List[str]] = [
    ["neutral"],
    ["troll"],
    ["dystopian"],
    ["devilsadvocate"],
    ["know-it-all"],
    ["mediator"],
    ["educator"],
    ["karen"],
    ["misogynist"],
    ["esoteric"],
    ["hippie-dense"],
    ["supportive-saviour-dense"],
    ["influencer"],
    ["social-media-activist-dense"],
    ["anti"],
    ["hobby-expert-dense"],
    ["narcissist"],
    ["sceptic"],
    ["shy"],
    ["toxic-positive"],
    ["traditionalist"]
]

languagelist = ["language-youth", "language-default", "language-old"]

political_viewlist = ["conservative",
                     "disengaged",
                     "far-right",
                     "progressive",
                     "radical-left"
]

def find_language(persona):
    for language in languagelist:
        if language in persona:
            return language
    return None
dataset['language_prompt'] = dataset['persona'].apply(find_language)

def find_main_persona(persona):
    for p in personaslist:
        if p[0] in persona:
            return p[0]
    return None
dataset['main_persona'] = dataset['persona'].apply(find_main_persona)

def find_background(persona):
    for background in backgroundlist:
        if background in persona:
            return background
    return None
dataset['background'] = dataset['persona'].apply(find_background)
def find_political_view(persona):
    for political_view in political_viewlist:
        if political_view in persona:
            return political_view
    return None
dataset['political_view'] = dataset['persona'].apply(find_political_view)
dataset
#grouped_dataset = dataset[dataset['persona'].str.contains('language-old')].groupby('persona')
#grouped_dataset[:2]
#    dataset['background'] = [p[0] for p in persona]
#    dataset['main_persona'] = [p[1] for p in persona]


,persona,model,original_author,threads,topic,text,payload.id,payload.timestamp,payload.action,payload.persona.id,...,payload.prompt,payload.response,annotation.politics,annotation.persona,annotation.tone,annotation.authenticity,language_prompt,main_persona,background,political_view
id,,,,,,,,,,,,,,,,,,,,,
b5fbd63f25b87a8d98ea494611916f08f20103711e4c9aec,teenager_misogynist-dense_language-youth,gpt-4-turbo-preview,conservative,Biden’s set on taking away your choice and for...,Environment,"bro, exactly, it's like they want us in chains...",29c97810-c691-11ee-b50d-28cfe9139b7b,2024-02-08T15:48:46.869125,reply,teenager_misogynist-dense_language-youth,...,Act as a Twitter user. Engage in political and...,"bro, exactly, it's like they want us in chains...",None,None,None,None,language-youth,misogynist,teenager,None
47af351ad1405b16dcc1b6750a530303677615f9c294fe82,affluent-young-woman_social-media-activist-den...,gpt-4-turbo-preview,alt_right,Ridiculous how our energy prices keep rising w...,Ukraine War,@alt_right it's wild you mention prioritizing ...,b83b9ea4-c68f-11ee-b50d-28cfe9139b7b,2024-02-08T15:38:26.860236,reply,affluent-young-woman_social-media-activist-den...,...,Act as a Twitter user. Engage in political and...,@alt_right it's wild you mention prioritizing ...,None,None,None,None,language-default,social-media-activist-dense,affluent-young-woman,radical-left


In [154]:
open(f'{DATA_OUT_PATH}.full.json', 'w').write(json.dumps(raw_samples, indent=4, ensure_ascii=False))

663080

In [155]:
for lang in LANGUAGES:
    open(f'{DATA_OUT_PATH}.{lang.lower()}.json', 'w').write(
        json.dumps(
            [record for record in raw_samples if record['language'] == lang], 
            indent=4, 
            ensure_ascii=False
        )
    )